In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('int_id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    borehole = Column(String(32), ForeignKey('Boreholes.id'),
                           primary_key=True)
    # Note that this could be a numeric ID as well
    id = Column(Integer, primary_key=True)
    description = Column(String(32))

In [3]:
engine = create_engine('sqlite:///test_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-04 11:18:49,944 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-04 11:18:49,945 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 11:18:49,947 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-04 11:18:49,947 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 11:18:49,949 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-04 11:18:49,949 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 11:18:49,951 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-04 11:18:49,951 INFO sqlalchemy.engine.base.Engine ()


In [7]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [11]:
boreholes = [Borehole(id='F11'), Borehole(id='F12'), Borehole(id='F13')] 

In [12]:
boreholes[0].intervals_values = {0: 'stuff', 1: 'junk'}
boreholes[1].intervals_values = {0: 'foo', 1: 'bar'}

In [15]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)